# Chapter 11. 인터페이스 : 프로토콜에서 ABC까지 

- 덕 타이핑의 상징인 동적 프로토콜에서부터 인터페이스를 명시하고 구현이 인터페이스에 따르는지 검증하는 추상 베이스 클래스(Abstract Base Class)에 이르기까지 다양한 곳에서 인터페이스가 사용

### 11.1 파이썬 문화에서의 인터페이스와 프로토콜 

- 클래스가 상속하거나 구현한 공개 속성(메서드나 데이터 속성)들의 집합이 인터페이스이다.
- 여기에는 \_\_getitem__()이나 \_\_add__()같은 특별 메서드도 포함된다.
- 보호된 속성과 비공개 속성은 인터페이스에 속하지 않는다라고 정의.
- 공개된 데이터 속성을 객체의 인터페이스로 사용하는 것은 괜찮다,
- 인터페이스 정의 : 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부
- 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스 : 프로토콜. 프로토콜은 상속과 무관
- 클래스는 어떤 프로토콜을 구현해서 객체가 여러 역할을 할 수 있게 만들 수도 있다.

### 11.2 파이썬은 시퀀스를 찾아낸다

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()

In [3]:
f[1]

10

In [11]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

### 11.3 런타임에 프로토콜을 구현하는 멍키 패칭 

- 표준 random.shuffle() 함수는 다음과 같이 사용한다

In [5]:
from random import shuffle
l = list(range(10))
shuffle(l)

In [6]:
l

[2, 4, 3, 7, 5, 0, 6, 8, 1, 9]

In [12]:
deck = FrenchDeck()

In [13]:
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [14]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [15]:
FrenchDeck.__setitem__ = set_card

In [16]:
shuffle(deck)

In [17]:
deck[:5]

[Card(rank='A', suit='hearts'),
 Card(rank='8', suit='diamonds'),
 Card(rank='J', suit='hearts'),
 Card(rank='2', suit='clubs'),
 Card(rank='3', suit='clubs')]

- deck 객체에 \_cards라는 이름의 속성이 있고, \_cards가 가변 시퀀스임을 set_card()가 알고 있다는 것이 비결
- 그러고 나서 set_card() 함수가 FrenchDeck 클래스의 \_\_setitem__ 특별 메서드에 연결된다.
- 이 방법은 멍키 패팅 (Monkey patching)의 한 예
- 멍키 패칭은 소스코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위

### 11.4 알렉스 마르텔리의 물새 

- 덕 타이핑은 객체의 실제 자료형은 무시하고, 대신 객체가 용도에 맞는 메서드 이름, 시그너처, 의미를 구현하도록 보장해준다.
- 구스 타이핑은 cls가 추상 베이스 클래스인 경우, 즉 cls의 메타클래스가 abc.ABCMeta인 경우에는 isinstance(obj, cls)를 써도 좋다는 의미.

In [19]:
class Struggle:
    def __len__(self):
        return 23

In [20]:
from collections import abc

In [21]:
isinstance(Struggle(), abc.Sized)

True

- numbers, collections.abc, 혹은 여러분이 사용할 다른 프레임워크에 있는 ABC가 표현하는 개념을 실현하는 클래스를 구현할 때는 언제다 해당 ABC를 상속하거나 해당 ABC를 등록하라.
- 배포용 코드에서 절대로 ABC나 메타클래스를 직접 구현하지 말라.

### 11.5 ABC 상속하기

In [22]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)

### 11.6 표준 라이브러리의 ABC 

##### 11.6.1 collections.abc의 ABC

- 파이썬 3.4 버전의 collection.abc에 정의된 16개의 ABC 속성명을 생략하고 간단히 UML 클래스 다이어그램으로 보여주고 있음.
- ABC에서는 일반적으로 다중 상속이 문제되지 않는다.

- Iterable, Container, Sized : 모든 컬렉션은 이 ABC를 상속하거나, 적어도 호환되는 프로토콜을 구현해야 한다. Iterrable은 \_\_iter__()를 통해 반복을, Container는 \_\_contains__()를 통해 in 연산자를, Sized는 \_\_len__()을 통해 len()메서드를 지원한다.
- Sequence, Mapping, Set : 주요 불변 컬렉션형으로서, 각기 가변형 서브 클래스가 있다.
- MappingView : 파이썬 3에서 items(), keys(), values() 메서드에서 반환된 객체는 각기 ItemsView, KeysView, ValuesView를 상속한다.
- Callable, Hashable : 이 두 ABC는 컬렉션과 밀접한 연관이 있는 것은 아니지만, collections.abc가 파이썬 표준 라이브러리 안에서 ABC를 정의한 최초의 패키지이다.
- Iterator : Iterator는 Iterable을 상속한다

##### 11.6.2 ABC의 숫자탑

- numbers 패키지는 소위 '숫자탑'이라고 하는것을 정의.
- Nnumber가 최상위 슈퍼 클래스이며 그 밑에 Complex, 그리고 Integeral까지 내려간다.

### 11.7 ABC의 정의와 사용

- 광고를 보여줄 떄 무작위 순으로 보여주지만 모든 광고를 다 보기 전까지 같은 광고를 반복하면 안된다.

In [25]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
    
    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 떄 이 매서드를 실행하면 'LookupError'가 발생한다."""
        
    def loaded(self):
        """최소 한 개의 항목이 있으면 True, 아니면 False를 반환한다."""
        return bool(self.inspect())
    
    def inspect():
        """현재 안에 있는 항목들로 구성된 정렬된 튜플을 반환한다."""
        item = []
        while True:
            try :
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [26]:
class Fake(Tombola):
    def pick(self):
        return 13

In [27]:
Fake

__main__.Fake

In [28]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

##### 11.7.1 ABC 상세 구문

- @abstractmethod 외에도 abc 모듈은 @abstractclassmethod, @abstractstaticmethod, @abstractproperty 데커레이터를 정의한다.

##### 11.7.2 Tombola ABC 상속하기

In [29]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()
            

In [ ]:
class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)

### 11.7.3 Tombola의 가상 서브클래스